In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector
from functools import reduce

### LECTURA DATOS

In [2]:
mysql_config = {
    'host': 'localhost',
    'user': 'root',
    'database': 'cancer_mama'
}

conn_mysql = mysql.connector.connect(**mysql_config,)
cursor_mysql = conn_mysql.cursor()

query = "SELECT * FROM tumor;"
df_sql = pd.read_sql_query(query, conn_mysql)

C:\Users\34699\AppData\Local\Temp\ipykernel_19988\276634330.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql_query(query, conn_mysql)


In [3]:
df_csv1 = pd.read_csv('./Datos/patients_batch1.csv')
df_csv2 = pd.read_csv('./Datos/patients_batch2.csv')
df_excel = pd.read_excel('./Datos/gynecological.xlsx')
df_json = pd.read_json('./Datos/histochemistry.json')

In [4]:
df_sql

,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,10011773,1,IS,0,0,None,None,None,0,None,1,NaN,NaN,no
1,10020495,1,1,0,0,None,None,None,IA,None,2,1.0,NaN,no
2,10020495,2,3,1,0,2,0,None,IA,None,2,1.0,NaN,yes
3,10030299,1,1,0,0,None,None,None,IA,None,1,1.0,NaN,no
4,10030824,1,2,1,0,2,2,0,IIIA,IIIA,3,NaN,1.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99966287,1,1,1,0,1,0,0,IB,IIA,3,1.0,NaN,yes
11163,99981192,1,1,0,0,None,None,None,IA,None,2,NaN,NaN,no
11164,99988958,1,0,2,0,1,1,0,IIIA,IIA,1,1.0,NaN,yes
11165,99992350,1,2,0,0,3,2,0,IIA,IIIA,2,1.0,NaN,yes


### PREPROCESADO

Para el preprocesado vamos a comenzar uniendo los dataframes de origen csv, ya que comparten las mismas columnas. Pasaremos las fechas al tipo 'DD/MM/YY' y a continuación haremos el merge.

In [48]:
dataframes = [df_json,df_csv2,df_csv1,df_sql,df_excel]
df_final = reduce(lambda left, right: pd.merge(left, right, on='ehr', how='outer'), dataframes)
df_final

In [60]:
df_csv = pd.merge(df_csv1, df_csv2, on='ehr', how='outer')
df_csv

,ehr,birth_date_x,diagnosis_date_x,death_date_x,birth_date_y,diagnosis_date_y,death_date_y
0,10011773,1959-07-05,2015-04-02,NaN,NaN,NaN,NaN
1,10020495,1953-10-02,2017-12-04,NaN,NaN,NaN,NaN
2,10030299,1966-08-16,2019-06-27,NaN,NaN,NaN,NaN
3,10030824,1953-03-03,2018-09-07,NaN,NaN,NaN,NaN
4,10041592,1959-07-06,2018-11-19,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
9995,99966287,NaN,NaN,NaN,16/12/1978,02/08/2018,NaN
9996,99981192,NaN,NaN,NaN,10/04/1949,28/10/2017,NaN
9997,99988958,NaN,NaN,NaN,21/01/1966,27/05/2013,NaN
9998,99992350,NaN,NaN,NaN,04/09/1939,17/12/2018,NaN
